# Importing Libraries

In [3]:
#importing technologies/libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import math 
import sys

In [4]:
#loading csv files into notebook
csv_files = glob("./zippedData/*.csv.gz") #
csv_files

['./zippedData\\bom.movie_gross.csv.gz',
 './zippedData\\imdb.name.basics.csv.gz',
 './zippedData\\imdb.title.akas.csv.gz',
 './zippedData\\imdb.title.basics.csv.gz',
 './zippedData\\imdb.title.crew.csv.gz',
 './zippedData\\imdb.title.principals.csv.gz',
 './zippedData\\imdb.title.ratings.csv.gz',
 './zippedData\\tmdb.movies.csv.gz',
 './zippedData\\tn.movie_budgets.csv.gz']

## Cleaning the file names

In [5]:
#renaming csv files to cleaner format
csv_files_dict = {}
for filename in csv_files:
    filename_cleaned = os.path.basename(filename).replace(".csv", "").replace(".", "_").replace("_gz","") 
    filename_df = pd.read_csv(filename, index_col=0)
    csv_files_dict[filename_cleaned] = filename_df

In [6]:
#creating a dictionary with keys as file names
csv_files_dict.keys()

dict_keys(['bom_movie_gross', 'imdb_name_basics', 'imdb_title_akas', 'imdb_title_basics', 'imdb_title_crew', 'imdb_title_principals', 'imdb_title_ratings', 'tmdb_movies', 'tn_movie_budgets'])

In [7]:
#declaring variables for each file imported file
bom_movie_gross = csv_files_dict['bom_movie_gross']
imdb_name_basics = csv_files_dict['imdb_name_basics']
imdb_title_akas = csv_files_dict['imdb_title_akas']
imdb_title_basics = csv_files_dict['imdb_title_basics']
imdb_title_crew = csv_files_dict['imdb_title_crew']
imdb_title_principals = csv_files_dict['imdb_title_principals']
imdb_title_ratings = csv_files_dict['imdb_title_ratings']
tmdb_movies = csv_files_dict['tmdb_movies']
movie_budgets = csv_files_dict['tn_movie_budgets']
rt_movie = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter = '\t', encoding = 'unicode_escape')
 #data copied and pasted from "https://www.the-numbers.com/"
top_producers_intl = pd.read_csv('databases/top_producers_intl.csv', encoding='cp1252')
genre_market_share = pd.read_csv('databases/genre_market_share.csv', encoding='cp1252')
rating_market_share = pd.read_csv('databases/rating_market_share.csv', encoding='cp1252')
#data copied and pasted to csv from wikipedia
franchises = pd.read_csv('databases/franchises_clean.csv')

# Cleaning each database

## Box Office Mojo

In [97]:
#getting data types and null counts
bom_movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3356 entries, Toy Story 3 to An Actor Prepares
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   studio          3356 non-null   object 
 1   domestic_gross  3356 non-null   float64
 2   year            3356 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 104.9+ KB


In [98]:
#visually inspecting the data
bom_movie_gross.head()

,studio,domestic_gross,year
title,,,
Toy Story 3,BV,415000000.0,2010
Alice in Wonderland (2010),BV,334200000.0,2010
Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,2010
Inception,WB,292600000.0,2010
Shrek Forever After,P/DW,238700000.0,2010


In [10]:
# evaluating if it is appropriate to remove 'foreign_gross' column
print('Percentage of Null foreign_gross values:', 
      len(bom_movie_gross[bom_movie_gross.foreign_gross.isna()])/ len(bom_movie_gross))

Percentage of Null foreign_gross values: 0.3985828166519043


We are going to remove the foreign_gross column as 40% of values are null

In [11]:
#code to remove the 'foreign_gross' column
bom_movie_gross = bom_movie_gross.drop(['foreign_gross'], axis = 1)

In [12]:
# Checkoint for data types and null counts
bom_movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3387 entries, Toy Story 3 to An Actor Prepares
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   studio          3382 non-null   object 
 1   domestic_gross  3359 non-null   float64
 2   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 105.8+ KB


Still need to remove null values from other columns

In [13]:
#decided to remove remaininng rows containing null values
bom_movie_gross = bom_movie_gross.dropna()
bom_movie_gross.isna().sum()

studio            0
domestic_gross    0
year              0
dtype: int64

In [14]:
#looks good, so exporting clean data to new file
bom_movie_gross.to_csv('databases/bom_movie_gross_clean.csv')

Null values removed! Data cleaned in bom_movie_gross_gz

## The Numbers: Movie Budgets

In [15]:
#Initial visual expection of the data
movie_budgets.head() 

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [17]:
#exploring data to look for redundant or missing values
movie_budgets.worldwide_gross.value_counts(normalize= False)

$0              367
$8,000,000        9
$2,000,000        6
$7,000,000        6
$4,000,000        4
               ... 
$52,460,543       1
$367,582          1
$128,885,873      1
$4,495,262        1
$247,812,011      1
Name: worldwide_gross, Length: 5356, dtype: int64

In [16]:
#examining the data with 0's in domestic gross, as MSFT is not interested in movies just released abroad
movie_budgets.sort_values(by='domestic_gross')

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
82,"Oct 26, 2012",Mientras duermes,"$5,000,000",$0,"$9,109,597"
9,"Nov 4, 2005",Wal-Mart: The High Cost of Low Price,"$1,500,000",$0,"$58,692"
8,"Jul 20, 2018",Teefa in Trouble,"$1,500,000",$0,"$98,806"
7,"Oct 17, 2014",Housebound,"$1,500,000",$0,"$236,863"
58,"Dec 31, 2008",Bathory,"$15,000,000",$0,"$3,436,763"
...,...,...,...,...,...
8,"Jun 13, 1997",Hercules,"$70,000,000","$99,112,101","$250,700,000"
74,"Jun 4, 1999",Desert Blue,"$5,000,000","$99,147","$99,147"
43,"Aug 3, 2018",Christopher Robin,"$75,000,000","$99,215,042","$197,504,758"


Looks like there are a lot of $0's will remove these as movies with no revenue are not of interest

In [104]:
#exploring data to look for redundant or missing values
movie_budgets.production_budget.value_counts(normalize= False)

20000000     221
10000000     197
30000000     171
40000000     163
15000000     162
            ... 
420000         1
7303082        1
306000000      1
4638783        1
28500000       1
Name: production_budget, Length: 472, dtype: int64

Looks like estimates or rounding is used creating multiple entries at the same value.  This is acceptible.

In [100]:
#exploring data to look for redundant or missing values
movie_budgets.domestic_gross.value_counts(normalize= False)

8000000      9
2000000      7
7000000      7
10000000     6
11000000     5
            ..
449865       1
10547117     1
4476235      1
56653        1
130179072    1
Name: domestic_gross, Length: 5163, dtype: int64

In [20]:
#examining data types
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5782 entries, 1 to 82
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5782 non-null   object
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   object
 3   domestic_gross     5782 non-null   object
 4   worldwide_gross    5782 non-null   object
dtypes: object(5)
memory usage: 271.0+ KB


Need to convert financial data to Int data type to enable statistical calculations

In [99]:
#First create a function to remove '$', ',' and change to int data type
def convert_amt_to_int(df, col):
    df[col] = df[col].str.replace("$", "").str.replace(",", "").astype('int64')
    return df

In [22]:
#Declare variable with columns to be altered
money_cols = ['production_budget', 'domestic_gross', 'worldwide_gross']
#input the column variable into the preceeding function
for col in money_cols:
    movie_budgets = convert_amt_to_int(movie_budgets, col)

In [105]:
# remove '0' values from the financial data
movie_budgets = movie_budgets[movie_budgets['domestic_gross'] > 0]
movie_budgets = movie_budgets[movie_budgets['production_budget'] > 0]

In [106]:
#data looks as desired
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5234 entries, 1 to 82
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5234 non-null   object
 1   movie              5234 non-null   object
 2   production_budget  5234 non-null   int64 
 3   domestic_gross     5234 non-null   int64 
 4   worldwide_gross    5234 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 245.3+ KB


In [25]:
#looks good, so exporting clean data to new file
movie_budgets.to_csv('databases/movie_budgets_clean.csv')

## The Numbers: Top Producers International

This data was retrieved by our team from:
https://www.the-numbers.com/box-office-star-records/domestic/lifetime-specific-technical-role/producer
We copied the data into a csv file using copy and paste

In [26]:
#Initial visual expection of the data
top_producers_intl.head()

,rank,name,interantional,movies,average
0,1,Kevin Feige,1.403169e+10,30.0,467723155.0
1,2,David Heyman,7.867445e+09,26.0,302594047.0
2,3,Kathleen Kennedy,6.903513e+09,35.0,197243218.0
3,4,Jerry Bruckheimer,6.520507e+09,36.0,181125190.0
4,5,Neal H. Moritz,6.049582e+09,49.0,123460866.0


In [27]:
#examining data types and looking for null values
top_producers_intl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rank             500 non-null    int64  
 1   name             500 non-null    object 
 2    interantional   500 non-null    float64
 3    movies          500 non-null    float64
 4    average         500 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 19.7+ KB


## The Numbers: Genre Market Share

This data was retrieved by our team from:
https://www.the-numbers.com/box-office-star-records/domestic/lifetime-specific-technical-role/producer
We copied the data into a csv file using copy and paste

In [28]:
#Initial visual expection of the data
genre_market_share.head()

,rank,genre,movies,total_box_office,tickets,share
0,1,Adventure,1056.0,6.369154e+10,9.114757e+09,0.27
1,2,Action,1049.0,4.772832e+10,6.766371e+09,0.20
2,3,Drama,5370.0,3.738627e+10,5.723810e+09,0.16
3,4,Comedy,2350.0,3.419589e+10,5.261717e+09,0.15
4,5,Thriller/Suspense,1110.0,1.971082e+10,2.870965e+09,0.08


In [29]:
#examining data types and looking for null values
genre_market_share.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              14 non-null     int64  
 1   genre             14 non-null     object 
 2   movies            14 non-null     float64
 3   total_box_office  14 non-null     float64
 4   tickets           14 non-null     float64
 5   share             14 non-null     object 
dtypes: float64(3), int64(1), object(2)
memory usage: 800.0+ bytes


Data is clean to need to alter

## The Numbers: Ratings Market Share


This data was retrieved by our team from: 
https://www.the-numbers.com/market/mpaa-ratings 
We copied the data into a csv file using copy and paste

In [30]:
#Initial visual expection of the data
rating_market_share.head()

,rank,rating,movies,total_box_office,tickets,share
0,1,PG-13,3143.0,1.113364e+11,1.604386e+10,0.48
1,2,R,5329.0,6.274039e+10,9.569086e+09,0.27
2,3,PG,1478.0,4.861201e+10,6.958419e+09,0.21
3,4,G,387.0,9.579448e+09,1.534342e+09,0.04
4,5,Not Rated,5815.0,1.917703e+09,2.850633e+08,0.01


In [31]:
#examining data types and looking for null values
ating_market_share.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              9 non-null      int64  
 1   rating            9 non-null      object 
 2   movies            9 non-null      float64
 3   total_box_office  9 non-null      float64
 4   tickets           9 non-null      float64
 5   share             9 non-null      float64
dtypes: float64(4), int64(1), object(1)
memory usage: 560.0+ bytes


Data is clean to need to alter

## IMDB "Name Basics"

In [107]:
#Initial visual expection of the data
imdb_name_basics.head()

,primary_name,primary_profession,known_for_titles
nconst,,,
nm0061671,Mary Ellen Bauder,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
nm0061865,Joseph Bauer,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
nm0062070,Bruce Baum,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
nm0062195,Axel Baumann,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
nm0062798,Pete Baxter,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [33]:
line_1 =imdb_name_basics.iloc[0]

In [34]:
def print_line(row):
    list_of_rows=[]
    for title_id in row['known_for_titles']:
        list_of_rows.append((row[['nconst', 'primary_profession', 'known_for_title']]))
        

In [35]:
#examining data types and looking for null values
imdb_name_basics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 606648 entries, nm0061671 to nm9993380
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   primary_name        606648 non-null  object 
 1   birth_year          82736 non-null   float64
 2   death_year          6783 non-null    float64
 3   primary_profession  555308 non-null  object 
 4   known_for_titles    576444 non-null  object 
dtypes: float64(2), object(3)
memory usage: 27.8+ MB


In [36]:
# evaluating if it is appropriate to remove 'birth_year' column
print('Percentage of Null birth_year Values:', len(imdb_name_basics[imdb_name_basics.birth_year.isna()])/ len(imdb_name_basics))

Percentage of Null birth_year Values: 0.8636177816460286


In [37]:
# evaluating if it is appropriate to remove 'death_year' column
print('Percentage of Null death_year Values:', len(imdb_name_basics[imdb_name_basics.death_year.isna()])/ len(imdb_name_basics))

Percentage of Null death_year Values: 0.9888188867349764


In [38]:
#removing birth year column
imdb_name_basics = imdb_name_basics.drop('birth_year', axis = 1)

In [39]:
#removing death year column
imdb_name_basics = imdb_name_basics.drop('death_year', axis = 1)

In [40]:
#Checkpoint to reexamine the data for null objects
imdb_name_basics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 606648 entries, nm0061671 to nm9993380
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   primary_name        606648 non-null  object
 1   primary_profession  555308 non-null  object
 2   known_for_titles    576444 non-null  object
dtypes: object(3)
memory usage: 18.5+ MB


In [41]:
#decided to remove rows with null values for the remaining series.  
imdb_name_basics = imdb_name_basics.dropna()
imdb_name_basics.isna().sum()

primary_name          0
primary_profession    0
known_for_titles      0
dtype: int64

In [112]:
#another visual inspection
imdb_name_basics.head()

,primary_name,primary_profession,known_for_titles
nconst,,,
nm0061671,Mary Ellen Bauder,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
nm0061865,Joseph Bauer,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
nm0062070,Bruce Baum,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
nm0062195,Axel Baumann,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
nm0062798,Pete Baxter,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [43]:
#looks good, so exporting clean data to new file
imdb_name_basics.to_csv('databases/imdb_name_basics_clean.csv')

## IMDB "Title Akas"

In [44]:
#Initial visual expection of the data
imdb_title_akas.sort_values(by='region')

,ordering,title,region,language,types,attributes,is_original_title
title_id,,,,,,,
tt3156000,3,Nick,AD,NaN,NaN,NaN,0.0
tt6079382,1,Impacto,AD,NaN,NaN,NaN,0.0
tt5162282,1,73',AD,NaN,NaN,NaN,0.0
tt1811329,2,"Teta, Alf Marra",AE,NaN,NaN,NaN,0.0
tt7480896,1,Desert Dream,AE,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...
tt9723084,2,Anderswo. Allein in Afrika,NaN,NaN,original,NaN,1.0
tt9726638,2,Monkey King: The Volcano,NaN,NaN,original,NaN,1.0
tt9755806,3,Big Shark,NaN,NaN,original,NaN,1.0


In [45]:
#examining data types and looking for null values
imdb_title_akas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331703 entries, tt0369610 to tt9880178
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ordering           331703 non-null  int64  
 1   title              331703 non-null  object 
 2   region             278410 non-null  object 
 3   language           41715 non-null   object 
 4   types              168447 non-null  object 
 5   attributes         14925 non-null   object 
 6   is_original_title  331678 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 20.2+ MB


In [46]:
# evaluating if it is appropriate to remove 'language' column
print('Percentage of Null death_year Values:', len(imdb_title_akas[imdb_title_akas.language.isna()])/ len(imdb_title_akas))
#answer is yes

Percentage of Null death_year Values: 0.87423990738703


In [47]:
# evaluating if it is appropriate to remove 'attributes' column
print('Percentage of Null death_year Values:', len(imdb_title_akas[imdb_title_akas.attributes.isna()])/ len(imdb_title_akas))
#answer is yes

Percentage of Null death_year Values: 0.9550049291082685


In [48]:
# evaluating if it is appropriate to remove 'types' column
print('Percentage of Null death_year Values:', len(imdb_title_akas[imdb_title_akas.types.isna()])/ len(imdb_title_akas))
#answer is yes

Percentage of Null death_year Values: 0.49217522904526034


In [49]:
#removing the 'types' column
imdb_title_akas = imdb_title_akas.drop('types', axis = 1)

In [50]:
#removing the 'attributes' column
imdb_title_akas = imdb_title_akas.drop('attributes', axis = 1)

In [51]:
#removing the 'language' column
imdb_title_akas = imdb_title_akas.drop('language', axis = 1)

In [52]:
#Checkpoint to reexamine the data for null objects
imdb_title_akas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331703 entries, tt0369610 to tt9880178
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ordering           331703 non-null  int64  
 1   title              331703 non-null  object 
 2   region             278410 non-null  object 
 3   is_original_title  331678 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 12.7+ MB


In [53]:
#Filling the 'is_original_title' column null values with '0' values as we aren't
#willing to drop those rows.
imdb_title_akas['is_original_title'] = imdb_title_akas['is_original_title'].fillna(value = 0)

In [54]:
#Filling the 'region' column null values with 'no region' as we aren't will to drop those rows
imdb_title_akas['region'] = imdb_title_akas['region'].fillna(value= 'no region')

In [113]:
#Checkpoint to reexamine the data for null objects
imdb_title_akas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331703 entries, tt0369610 to tt9880178
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ordering           331703 non-null  int64  
 1   title              331703 non-null  object 
 2   region             331703 non-null  object 
 3   is_original_title  331703 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 12.7+ MB


In [55]:
#looks good, so exporting clean data to new file
imdb_title_akas.to_csv('databases/imdb_title_akas_clean.csv')

## IMDB "Title Crew"

In [108]:
#Initial visual expection of the data
imdb_title_crew.head()

,directors,writers
tconst,,
tt0285252,nm0899854,nm0899854
tt0438973,unknown,"nm0175726,nm1802864"
tt0462036,nm1940585,nm1940585
tt0835418,nm0151540,"nm0310087,nm0841532"
tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


In [57]:
#examining data types and looking for null values
imdb_title_crew.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146144 entries, tt0285252 to tt9010172
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   directors  140417 non-null  object
 1   writers    110261 non-null  object
dtypes: object(2)
memory usage: 3.3+ MB


In [58]:
#determining if there is overlap
imdb_title_crew.isna().sum()

directors     5727
writers      35883
dtype: int64

In [114]:
#there is overlap and we are unwilling to remove 35k rows, going to fill null values with 'unknown'
imdb_title_crew['writers'] = imdb_title_crew['writers'].fillna(value= 'unknown')
imdb_title_crew['directors'] = imdb_title_crew['directors'].fillna(value= 'unknown')

In [60]:
#Checkpoint to reexamine the data for null objects
imdb_title_crew.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146144 entries, tt0285252 to tt9010172
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   directors  146144 non-null  object
 1   writers    146144 non-null  object
dtypes: object(2)
memory usage: 3.3+ MB


In [61]:
#looks good, so exporting clean data to new file
imdb_title_crew.to_csv('databases/imdb_title_crew_clean.csv')

## IMDB "Title Principals"

In [62]:
#Initial visual expection of the data
imdb_title_principals.head()

,ordering,nconst,category,job,characters
tconst,,,,,
tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
tt0111414,2,nm0398271,director,NaN,NaN
tt0111414,3,nm3739909,producer,producer,NaN
tt0323808,10,nm0059247,editor,NaN,NaN
tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


In [63]:
#examining data types and looking for null values
imdb_title_principals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028186 entries, tt0111414 to tt9692684
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   ordering    1028186 non-null  int64 
 1   nconst      1028186 non-null  object
 2   category    1028186 non-null  object
 3   job         177684 non-null   object
 4   characters  393360 non-null   object
dtypes: int64(1), object(4)
memory usage: 47.1+ MB


In [64]:
imdb_title_principals = imdb_title_principals.drop('job', axis = 1)
# dropped job because it was redundent of catagory

In [65]:
# want to keep the character column and null rows, going to fill null values with 'not applicable'
imdb_title_principals['characters'] = imdb_title_principals['characters'].fillna(value= 'not applicable')

In [66]:
#Checkpoint to reexamine the data for null objects
imdb_title_principals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028186 entries, tt0111414 to tt9692684
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   ordering    1028186 non-null  int64 
 1   nconst      1028186 non-null  object
 2   category    1028186 non-null  object
 3   characters  1028186 non-null  object
dtypes: int64(1), object(3)
memory usage: 39.2+ MB


In [67]:
#looks good, so exporting clean data to new file
imdb_title_principals.to_csv('databases/imdb_title_principals_clean.csv')

## IMDB "Title Ratings"

In [109]:
#Initial visual expection of the data
imdb_title_ratings.head()

,averagerating,numvotes
tconst,,
tt10356526,8.3,31
tt10384606,8.9,559
tt1042974,6.4,20
tt1043726,4.2,50352
tt1060240,6.5,21


In [69]:
#examining data types and looking for null values
imdb_title_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73856 entries, tt10356526 to tt9894098
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   averagerating  73856 non-null  float64
 1   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1)
memory usage: 1.7+ MB


In [70]:
#examning value counts, nothing out of the ordinary
imdb_title_ratings.averagerating.value_counts(normalize= False)

7.0     2262
6.6     2251
7.2     2249
6.8     2239
6.5     2221
        ... 
9.6       18
10.0      16
9.8       15
9.7       12
9.9        5
Name: averagerating, Length: 91, dtype: int64

In [115]:
#examning value counts, nothing out of the ordinary
imdb_title_ratings.numvotes.value_counts(normalize= False)

6        2875
5        2699
7        2476
8        2167
9        1929
         ... 
18286       1
16289       1
1958        1
4007        1
4098        1
Name: numvotes, Length: 7349, dtype: int64

In [72]:
#looks good, so exporting clean data to new file
imdb_title_ratings.to_csv('databases/imdb_title_ratings_clean.csv')

## The Movie DB "Movies"

In [110]:
#Initial visual expection of the data
tmdb_movies.head()

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [74]:
#examining data types and looking for null values
tmdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB


In [116]:
#lots of columns to examine with value_counts so we are writing a function
list_of_values= []
def value_count(df):
    for header in df: #iterates through each column with value_counts
         list_of_values.append(df[header].value_counts(normalize= False))
    

In [117]:
#using function to look for missing or redundant data
value_count(tmdb_movies)
list_of_values

[[18]                         962
 [99]                         668
 [35]                         635
 [27]                         316
 [35, 18]                     251
                             ... 
 [14, 28, 878]                  1
 [12, 14, 27, 28, 53, 878]      1
 [16, 12, 10749]                1
 [53, 18, 14, 27]               1
 [28, 53, 10752, 18]            1
 Name: genre_ids, Length: 1729, dtype: int64,
 121803    3
 192767    3
 152042    3
 244539    3
 228108    3
          ..
 51764     1
 82485     1
 245855    1
 520370    1
 524288    1
 Name: id, Length: 9349, dtype: int64,
 en    8301
 fr     337
 es     195
 ja     182
 hi     132
 ru     111
 de     106
 zh      91
 ko      57
 it      56
 sv      45
 pt      43
 da      35
 cn      31
 nl      31
 no      31
 pl      24
 he      22
 th      18
 ar      15
 fa      13
 tr      11
 ro      10
 is       8
 te       7
 id       7
 fi       7
 el       6
 hu       6
 sr       5
 ta       4
 uk       4
 tl       4
 x

In [77]:
#removing rows from data where the 'vote_count' is less than 10
tmdb_movies = tmdb_movies[tmdb_movies['vote_count'] > 10]
#validaty of rating is questionable with less than 10 votes

In [78]:
#looks good, so exporting clean data to new file
tmdb_movies.to_csv('databases/tmdb_movies_clean.csv')

## Rotten Tomatoes "Movie"

In [111]:
#Initial visual expection of the data
rt_movie.head()

,id,synopsis,rating,genre,director,writer,theater_date
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971"
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012"
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996"
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994"
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN


In [81]:
rt_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [82]:
#Removing columns that are not of interest, we plan to merge this dataframe with the 
#movie_budget dataframe so we don't need to box office here
rt_movie = rt_movie.drop(['currency'], axis = 1)
rt_movie = rt_movie.drop(['box_office'], axis = 1)
rt_movie = rt_movie.drop(['dvd_date'], axis = 1)
rt_movie = rt_movie.drop(['runtime'], axis = 1)
rt_movie = rt_movie.drop(['studio'], axis = 1)
rt_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
dtypes: int64(1), object(6)
memory usage: 85.4+ KB


In [83]:
# Will to accept null values in this dataframe as we will be mergine with movie_budgets
rt_movie.to_csv('databases/rt_movie_clean.csv')

## Franchise Databases

In [87]:
franchises

,Rank,Franchise,Total Gross,Movie Count,Average Gross
0,1,Marvel Cinematic Universe,"8,525",23,371
1,2,Star Wars,"4,221",11,384
2,3,Disney Live Action Reimaginings,"3,220",14,230
3,4,Wizarding World,"2,785",10,253
4,5,Avengers,"2,619",4,655
...,...,...,...,...,...
62,63,The Secret Life of Pets,525,2,263
63,64,"Monsters, Inc.",524,2,262
64,65,Kung Fu Panda,524,3,175
65,66,Rush Hour,508,3,169


In [88]:
franchises.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Rank             67 non-null     int64 
 1   Franchise        67 non-null     object
 2    Total Gross     67 non-null     object
 3    Movie Count     67 non-null     int64 
 4    Average Gross   67 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 2.7+ KB


In [89]:
franchises = franchises.set_index('Rank')

In [90]:
#looks good, so exporting clean data to new file
franchises.to_csv('databases/franchises_clean.csv')

In [91]:
extfranchises = pd.read_csv('databases/extfranchises_clean.csv', encoding='cp1252')

In [92]:
extfranchises

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Franchise,#movies,domesticboxoffice,infl_adj_box_office,worldwideboxoffice,firstyear,last year,#years
0,0,0,0,0,0,0,0,0,Marvel Cinematic Universe,31,8545426433,9102945801,22576311289,2008,2022,14
1,1,1,1,1,1,1,1,1,Star Wars,15,5079119579,8039797255,10323355075,1977,2026,49
2,2,2,2,2,2,2,2,2,Harry Potter,12,2786938291,3638704111,9195390708,2001,2021,20
3,3,3,3,3,3,3,3,3,Batman,22,2786778902,3949355696,6049670921,1989,2021,32
4,4,4,4,4,4,4,4,4,Spider-Man,11,2711115724,3292639145,7219770479,2002,2019,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,673,673,673,673,673,673,673,673,Django,2,19391,22137,19391,1966,2020,54
674,674,674,674,674,674,674,674,674,St. Trinian's,2,15000,18220,29830239,2009,2009,
675,675,675,675,675,675,675,675,675,The Wedding Party,2,11140,11140,73115,2017,2017,
676,676,676,676,676,676,676,676,676,Donald Strachey,2,4269,5402,4269,2008,2008,


In [93]:
extfranchises.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                678 non-null    int64 
 1   Unnamed: 0.1              678 non-null    int64 
 2   Unnamed: 0.1.1            678 non-null    int64 
 3   Unnamed: 0.1.1.1          678 non-null    int64 
 4   Unnamed: 0.1.1.1.1        678 non-null    int64 
 5   Unnamed: 0.1.1.1.1.1      678 non-null    int64 
 6   Unnamed: 0.1.1.1.1.1.1    678 non-null    int64 
 7   Unnamed: 0.1.1.1.1.1.1.1  678 non-null    int64 
 8   Franchise                 678 non-null    object
 9   #movies                   678 non-null    int64 
 10  domesticboxoffice         678 non-null    int64 
 11  infl_adj_box_office       678 non-null    int64 
 12  worldwideboxoffice        678 non-null    int64 
 13  firstyear                 678 non-null    int64 
 14  last year                 

In [94]:
#looks good, so exporting clean data to new file
extfranchises.to_csv('databases/extfranchises_clean.csv')